In [1]:
import numpy as np
import pandas as pd


#최대 최소값이 각각 1,0 이 되도록 스케일링 해줌
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score,f1_score,log_loss,roc_auc_score
from sklearn import svm
#shuffle split cross validation : train set의 크기를 유연하게 조절 할때 사용됨
from sklearn.model_selection import train_test_split,ShuffleSplit,cross_val_score,GridSearchCV

In [2]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression,SGDClassifier
from os.path import join as opj
from matplotlib import pyplot as plt

from matplotlib.colors import Normalize

In [3]:
train = pd.read_json('./train.json')

In [4]:
train.head(5)

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
2,58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
3,4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
4,271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [5]:
train.band_1.head(5)

0    [-27.878360999999998, -27.15416, -28.668615, -...
1    [-12.242375, -14.920304999999999, -14.920363, ...
2    [-24.603676, -24.603714, -24.871029, -23.15277...
3    [-22.454607, -23.082819, -23.998013, -23.99805...
4    [-26.006956, -23.164886, -23.164886, -26.89116...
Name: band_1, dtype: object

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1604 entries, 0 to 1603
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1604 non-null   object
 1   band_1      1604 non-null   object
 2   band_2      1604 non-null   object
 3   inc_angle   1604 non-null   object
 4   is_iceberg  1604 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 62.8+ KB


In [7]:
train.inc_angle.replace({'na':np.nan},inplace=True)
train.drop(train[train['inc_angle'].isnull()].index,inplace=True)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1471 entries, 0 to 1508
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          1471 non-null   object 
 1   band_1      1471 non-null   object 
 2   band_2      1471 non-null   object 
 3   inc_angle   1471 non-null   float64
 4   is_iceberg  1471 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 69.0+ KB


In [9]:
X_HH_train = np.array([np.array(band).astype(np.float32) for band in train.band_1])
X_HV_train = np.array([np.array(band).astype(np.float32) for band in train.band_2])
X_angle_train=np.array([[np.array(angle).astype(np.float32) for angle in train.inc_angle]]).T

y_train=train.is_iceberg.values.astype(np.float32)
X_train=np.concatenate((X_HH_train,X_HV_train,X_angle_train), axis=1)
X_train.shape

(1471, 11251)

In [10]:
scaler = MaxAbsScaler()
X_train_maxabs = scaler.fit_transform(X_train)

In [11]:
# Create the SVM instance using Radial Basis Function (rbf) kernel
clf = svm.SVC(kernel='rbf',probability=False)
# Set the range of hyper-parameter we wanna use to tune our SVM classifier
C_range = [0.1,1,10,50,100]
gamma_range = [0.00001,0.0001,0.001,0.01,0.1]
param_grid_SVM = dict(gamma=gamma_range, C=C_range)
# set the gridsearch using 3-fold cross validation and 'ROC Area Under the Curve' as the cross validation score. 
grid = GridSearchCV(clf, param_grid=param_grid_SVM, cv=3,scoring='roc_auc')
grid.fit(X_train_maxabs, y_train)
print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 10, 'gamma': 0.01} with a score of 0.92
